In [15]:
import os
import sys
import pathlib
import click
import yaml

In [16]:
import tensorflow as tf
from tensorflow.keras.utils import Progbar
from model.dataset import Dataset
from model.fp.melspec.melspectrogram import get_melspec_layer
from model.fp.specaug_chain.specaug_chain import get_specaug_chain_layer
from model.fp.nnfp import get_fingerprinter
from model.fp.NTxent_loss_single_gpu import NTxentLoss
from model.fp.online_triplet_loss import OnlineTripletLoss
from model.fp.lamb_optimizer import LAMB


def build_fp(cfg):
    """ Build fingerprinter """
    # m_pre: log-power-Mel-spectrogram layer, S.
    m_pre = get_melspec_layer(cfg, trainable=False)

    # m_specaug: spec-augmentation layer.
    m_specaug = get_specaug_chain_layer(cfg, trainable=False)
    assert(m_specaug.bypass==False) # Detachable by setting m_specaug.bypass.

    # m_fp: fingerprinter g(f(.)).
    m_fp = get_fingerprinter(cfg, trainable=False)
    return m_pre, m_specaug, m_fp

In [17]:
def load_config(config_fname):
    config_filepath = '../config/' + config_fname + '.yaml'
    if os.path.exists(config_filepath):
        print(f'cli: Configuration from {config_filepath}')
    else:
        sys.exit(f'cli: ERROR! Configuration file {config_filepath} is missing!!')

    with open(config_filepath, 'r') as f:
        cfg = yaml.safe_load(f)
    return cfg

In [18]:
config:str = "default" 
cfg = load_config(config)
dataset = Dataset(cfg)

cli: Configuration from ../config/default.yaml


In [19]:
# Learning schedule
total_nsteps = cfg['TRAIN']['MAX_EPOCH'] * len(dataset.get_train_ds())
if cfg['TRAIN']['LR_SCHEDULE'].upper() == 'COS':
    lr_schedule = tf.keras.experimental.CosineDecay(
        initial_learning_rate=float(cfg['TRAIN']['LR']),
        decay_steps=total_nsteps,
        alpha=1e-06)
elif cfg['TRAIN']['LR_SCHEDULE'].upper() == 'COS-RESTART':
    lr_schedule = tf.keras.experimental.CosineDecayRestarts(
        initial_learning_rate=float(cfg['TRAIN']['LR']),
        first_decay_steps=int(total_nsteps * 0.1),
        num_periods=0.5,
        alpha=2e-06)
else:
    lr_schedule = float(cfg['TRAIN']['LR'])

# Optimizer
if cfg['TRAIN']['OPTIMIZER'].upper() == 'LAMB':
    opt = LAMB(learning_rate=lr_schedule)
elif cfg['TRAIN']['OPTIMIZER'].upper() == 'ADAM':
    opt = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
else:
    raise NotImplementedError(cfg['TRAIN']['OPTIMIZER'])

# Loss objects
if cfg['LOSS']['LOSS_MODE'].upper() == 'NTXENT': # Default
    loss_obj_train = NTxentLoss(
        n_org=cfg['BSZ']['TR_N_ANCHOR'],
        n_rep=cfg['BSZ']['TR_BATCH_SZ'] - cfg['BSZ']['TR_N_ANCHOR'],
        tau=cfg['LOSS']['TAU'])
    loss_obj_val = NTxentLoss(
        n_org=cfg['BSZ']['VAL_N_ANCHOR'],
        n_rep=cfg['BSZ']['VAL_BATCH_SZ'] - cfg['BSZ']['VAL_N_ANCHOR'],
        tau=cfg['LOSS']['TAU'])
elif cfg['LOSS']['LOSS_MODE'].upper() == 'ONLINE-TRIPLET': # Now-playing
    loss_obj_train = OnlineTripletLoss(
        bsz=cfg['BSZ']['TR_BATCH_SZ'],
        n_anchor=cfg['BSZ']['TR_N_ANCHOR'],
        mode = 'semi-hard',
        margin=cfg['LOSS']['MARGIN'])
    loss_obj_val = OnlineTripletLoss(
        bsz=cfg['BSZ']['VAL_BATCH_SZ'],
        n_anchor=cfg['BSZ']['VAL_N_ANCHOR'],
        mode = 'all', # use 'all' mode for validation
        margin=0.)
else:
    raise NotImplementedError(cfg['LOSS']['LOSS_MODE'])

In [20]:
opt, loss_obj_train

(<keras.src.optimizers.adam.Adam at 0x7f4a5cbd8d50>,
 <model.fp.NTxent_loss_single_gpu.NTxentLoss at 0x7f4aac64d7d0>)

In [21]:
m_pre, m_specaug, m_fp = build_fp(cfg)

In [22]:
#m_fp.load_weights('/home/rodrigo/Documents/neural-audio-fp/logs/checkpoint/CHECKPOINT_BSZ_120/')
# Setup do checkpoint manager
checkpoint_path = '/home/rodrigo/Documents/neural-audio-fp/logs/checkpoint/CHECKPOINT_BSZ_120/'
#optimizer = tf.keras.optimizers.Adam()

checkpoint = tf.train.Checkpoint(optimizer=opt, model=m_fp)
checkpoint_manager = tf.train.CheckpointManager(
    checkpoint, 
    checkpoint_path, 
    max_to_keep=3, 
    keep_checkpoint_every_n_hours=1
)

# Carregar o último checkpoint
checkpoint.restore(checkpoint_manager.latest_checkpoint).expect_partial()

if checkpoint_manager.latest_checkpoint:
    print(f'Checkpoint restaurado de {checkpoint_manager.latest_checkpoint}')
else:
    print('Nenhum checkpoint encontrado.')

Checkpoint restaurado de /home/rodrigo/Documents/neural-audio-fp/logs/checkpoint/CHECKPOINT_BSZ_120/ckpt-100


In [23]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model, Input

def convlayer(hidden_ch=128,
              strides=[(1,1),(1,1)],
              norm='layer_norm2d'):
    conv2d_1x3 = tf.keras.layers.Conv2D(hidden_ch,
                                        kernel_size=(1, 3),
                                        strides=strides[0],
                                        padding='SAME',
                                        dilation_rate=(1, 1),
                                        kernel_initializer='glorot_uniform',
                                        bias_initializer='zeros')
    conv2d_3x1 = tf.keras.layers.Conv2D(hidden_ch,
                                        kernel_size=(3, 1),
                                        strides=strides[1],
                                        padding='SAME',
                                        dilation_rate=(1, 1),
                                        kernel_initializer='glorot_uniform',
                                        bias_initializer='zeros')
    if norm == 'layer_norm1d':
        BN_1x3 = tf.keras.layers.LayerNormalization(axis=-1)
        BN_3x1 = tf.keras.layers.LayerNormalization(axis=-1)
    elif norm == 'layer_norm2d':
        BN_1x3 = tf.keras.layers.LayerNormalization(axis=(1, 2, 3))
        BN_3x1 = tf.keras.layers.LayerNormalization(axis=(1, 2, 3))
    else:
        BN_1x3 = tf.keras.layers.BatchNormalization(axis=-1)
        BN_3x1 = tf.keras.layers.BatchNormalization(axis=-1)
        
    forward = tf.keras.Sequential([conv2d_1x3,
                                   tf.keras.layers.ELU(),
                                   BN_1x3,
                                   conv2d_3x1,
                                   tf.keras.layers.ELU(),
                                   BN_3x1
                                   ])
    
    return forward



def create_sequential_front_conv(input_shape=(256,32,1),
                                 emb_sz=128,
                                 front_hidden_ch=[128, 128, 256, 256, 512, 512, 1024, 1024],
                                 front_strides=[[(1,2), (2,1)], [(1,2), (2,1)],
                                                [(1,2), (2,1)], [(1,2), (2,1)],
                                                [(1,1), (2,1)], [(1,2), (2,1)],
                                                [(1,1), (2,1)], [(1,2), (2,1)]],
                                 norm='layer_norm2d'):
    front_conv = tf.keras.Sequential(name='ConvLayers')
    if ((front_hidden_ch[-1] % emb_sz) != 0):
        front_hidden_ch[-1] = ((front_hidden_ch[-1]//emb_sz) + 1) * emb_sz

    for i in range(len(front_strides)):
        front_conv.add(convlayer(hidden_ch=front_hidden_ch[i], strides=front_strides[i], norm=norm))
    front_conv.add(tf.keras.layers.Flatten())

    return front_conv



def auxiliar(input1):
    conv_layer = create_sequential_front_conv(input_shape=(256,32,1),
                                               emb_sz=128,
                                               front_hidden_ch=[128, 128, 256, 256, 512, 512, 1024, 1024],
                                               front_strides=[[(1,2), (2,1)], [(1,2), (2,1)],
                                                              [(1,2), (2,1)], [(1,2), (2,1)],
                                                              [(1,1), (2,1)], [(1,2), (2,1)],
                                                              [(1,1), (2,1)], [(1,2), (2,1)]],
                                               norm='layer_norm2d')

    unit_dim = [32, 1]
    q = 128
    arquiteturas_densas = tf.keras.Sequential([tf.keras.layers.Dense(unit_dim[0], activation='elu'),
                                               tf.keras.layers.Dense(unit_dim[1])])

    x = input1
    #x reshape
    x = conv_layer(x)

    y_list = [0] * q
    x_split = tf.split(x, num_or_size_splits=128, axis=1)

    for v, k in enumerate(x_split):
        y_list[v] = arquiteturas_densas(k)

    out = tf.concat(y_list, axis=1)
    output = tf.math.l2_normalize(out, axis=1)
    return output


def get_fingerprinting(input1):
    output = auxiliar(input1)
    fingerprinting_model = Model(inputs=input1, outputs=output)
    return fingerprinting_model

In [ ]:
m_fp.get_weights()

desenho do sistema, com os blocos de projeto.

o kapre num fluxo completo (modelo antigo)

dividir o projeto em blocos e depois substituir o que tenho
depois dentro dos blocos, os subblocos, gradio etc

subclassing -> kapre ->
functional -> triangular ->

predict, extraçao, faiss

base de dados, cap. resultados, o que preciso
introdução, ver as comparações todas
desenho e explicação dos blocos do projeto
descrição do functional e do subclassing com o keras - breve revisão do keras.tensofrlow, existe tal e tal e vantagens e desvantagens, como é bom para o projeto, porque objetivo
sumário da tese - todos os titulos do capítulo (sections, subsections etc)

In [ ]:
# Transferir os pesos do modelo subclassed para o modelo funcional

functional_model=get_fingerprinting(input1)
functional_model.set_weights(m_fp.get_weights())